In [ ]:
%matplotlib inline

import refnx
from refnx.reflect import SLD
from refnx.analysis import Parameter
from hogben.models.samples import Sample, MagneticLayerSLD

def simple_sample():
    """Define a bilayer sample, and return the associated refnx model"""
    # Define sample

    layer1_thick = Parameter(50, 'Layer1 thickness', (30, 150))
    layer2_thick = Parameter(40, 'Layer2 thickness', (30, 150))
    layer1_rough = Parameter(5, 'Layer1 Roughness', (2, 10))
    layer2_rough = Parameter(6, 'Layer2 roughness', (2, 10))

    ul_thick = Parameter(35, 'Underlayer Thickness', (35, 350))
    ul_nsld = Parameter(2, 'Underlayer Nuclear SLD', (1.5, 7))
    ul_msld = Parameter(2, 'Underlayer Magnetic SLD', (0, 6))    

    air = refnx.reflect.SLD(0, name='Air')    
    layer1 = refnx.reflect.SLD(3, name="Layer 1")(thick=layer1_thick, rough=layer1_rough)
    layer2 = refnx.reflect.SLD(-0.5, name="Layer 2")(thick=layer2_thick, rough=layer2_rough)

    underlayer = MagneticLayerSLD(SLDn=ul_nsld, SLDm=ul_msld, thick=ul_thick, rough=3, underlayer=True)    
    substrate = SLD(2.074, name='Substrate')(rough=3)


    params = [
        layer1_thick,
        layer2_thick,
        layer1_rough,
        layer2_rough,
    ]

    for param in params:
        param.vary = True

    optimize = [
        ul_msld,
        ul_thick,
        ul_nsld,
    ]

    for params in optimize:
        params.optimize = True

    sample = air | layer1  | layer2 | layer1 | underlayer | substrate
    return sample

sample = Sample(simple_sample())

angle_times = [(0.7, 100, 100),
               (2.3, 100, 400),
               ]


In [ ]:
sample.optimise_parameters(angle_times)

In [ ]:
sample.sld_profile(single=False)

In [ ]:
sample.reflectivity_profile()